# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f6d64084d00>
├── 'a' --> tensor([[-0.1308,  0.0565, -0.8242],
│                   [-0.6685, -1.1443,  1.0025]])
└── 'x' --> <FastTreeValue 0x7f6d64084c40>
    └── 'c' --> tensor([[-0.1607,  0.5661,  0.1415, -0.1729],
                        [-0.2221, -1.0410,  1.1426,  0.5805],
                        [-0.0153, -0.5120,  0.0779,  0.4097]])

In [4]:
t.a

tensor([[-0.1308,  0.0565, -0.8242],
        [-0.6685, -1.1443,  1.0025]])

In [5]:
%timeit t.a

62.7 ns ± 0.063 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f6d64084d00>
├── 'a' --> tensor([[ 0.3943,  1.4413, -1.1270],
│                   [-1.4557, -0.8792, -0.1550]])
└── 'x' --> <FastTreeValue 0x7f6d64084c40>
    └── 'c' --> tensor([[-0.1607,  0.5661,  0.1415, -0.1729],
                        [-0.2221, -1.0410,  1.1426,  0.5805],
                        [-0.0153, -0.5120,  0.0779,  0.4097]])

In [7]:
%timeit t.a = new_value

66.5 ns ± 0.0585 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.1308,  0.0565, -0.8242],
               [-0.6685, -1.1443,  1.0025]]),
    x: Batch(
           c: tensor([[-0.1607,  0.5661,  0.1415, -0.1729],
                      [-0.2221, -1.0410,  1.1426,  0.5805],
                      [-0.0153, -0.5120,  0.0779,  0.4097]]),
       ),
)

In [10]:
b.a

tensor([[-0.1308,  0.0565, -0.8242],
        [-0.6685, -1.1443,  1.0025]])

In [11]:
%timeit b.a

58 ns ± 0.137 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.0116,  1.7534, -0.9829],
               [-0.4864, -0.4339, -0.1054]]),
    x: Batch(
           c: tensor([[-0.1607,  0.5661,  0.1415, -0.1729],
                      [-0.2221, -1.0410,  1.1426,  0.5805],
                      [-0.0153, -0.5120,  0.0779,  0.4097]]),
       ),
)

In [13]:
%timeit b.a = new_value

498 ns ± 0.663 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

812 ns ± 0.177 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.9 µs ± 52.5 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

145 µs ± 330 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

146 µs ± 812 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f6caeaa1e20>
├── 'a' --> tensor([[[-0.1308,  0.0565, -0.8242],
│                    [-0.6685, -1.1443,  1.0025]],
│           
│                   [[-0.1308,  0.0565, -0.8242],
│                    [-0.6685, -1.1443,  1.0025]],
│           
│                   [[-0.1308,  0.0565, -0.8242],
│                    [-0.6685, -1.1443,  1.0025]],
│           
│                   [[-0.1308,  0.0565, -0.8242],
│                    [-0.6685, -1.1443,  1.0025]],
│           
│                   [[-0.1308,  0.0565, -0.8242],
│                    [-0.6685, -1.1443,  1.0025]],
│           
│                   [[-0.1308,  0.0565, -0.8242],
│                    [-0.6685, -1.1443,  1.0025]],
│           
│                   [[-0.1308,  0.0565, -0.8242],
│                    [-0.6685, -1.1443,  1.0025]],
│           
│                   [[-0.1308,  0.0565, -0.8242],
│                    [-0.6685, -1.1443,  1.0025]]])
└── 'x' --> <FastTreeValue 0x7f6cb556a550>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.8 µs ± 80 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f6d6714d760>
├── 'a' --> tensor([[-0.1308,  0.0565, -0.8242],
│                   [-0.6685, -1.1443,  1.0025],
│                   [-0.1308,  0.0565, -0.8242],
│                   [-0.6685, -1.1443,  1.0025],
│                   [-0.1308,  0.0565, -0.8242],
│                   [-0.6685, -1.1443,  1.0025],
│                   [-0.1308,  0.0565, -0.8242],
│                   [-0.6685, -1.1443,  1.0025],
│                   [-0.1308,  0.0565, -0.8242],
│                   [-0.6685, -1.1443,  1.0025],
│                   [-0.1308,  0.0565, -0.8242],
│                   [-0.6685, -1.1443,  1.0025],
│                   [-0.1308,  0.0565, -0.8242],
│                   [-0.6685, -1.1443,  1.0025],
│                   [-0.1308,  0.0565, -0.8242],
│                   [-0.6685, -1.1443,  1.0025]])
└── 'x' --> <FastTreeValue 0x7f6caeab93a0>
    └── 'c' --> tensor([[-0.1607,  0.5661,  0.1415, -0.1729],
                        [-0.2221, -1.0410,  1.1426,  0.5805],
                 

In [23]:
%timeit t_cat(trees)

30.4 µs ± 64.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

55.7 µs ± 56.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.1308,  0.0565, -0.8242],
                [-0.6685, -1.1443,  1.0025]],
       
               [[-0.1308,  0.0565, -0.8242],
                [-0.6685, -1.1443,  1.0025]],
       
               [[-0.1308,  0.0565, -0.8242],
                [-0.6685, -1.1443,  1.0025]],
       
               [[-0.1308,  0.0565, -0.8242],
                [-0.6685, -1.1443,  1.0025]],
       
               [[-0.1308,  0.0565, -0.8242],
                [-0.6685, -1.1443,  1.0025]],
       
               [[-0.1308,  0.0565, -0.8242],
                [-0.6685, -1.1443,  1.0025]],
       
               [[-0.1308,  0.0565, -0.8242],
                [-0.6685, -1.1443,  1.0025]],
       
               [[-0.1308,  0.0565, -0.8242],
                [-0.6685, -1.1443,  1.0025]]]),
    x: Batch(
           c: tensor([[[-0.1607,  0.5661,  0.1415, -0.1729],
                       [-0.2221, -1.0410,  1.1426,  0.5805],
                       [-0.0153, -0.5120,  0.0779,  0.4097]],
         

In [26]:
%timeit Batch.stack(batches)

80 µs ± 1.23 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.1308,  0.0565, -0.8242],
               [-0.6685, -1.1443,  1.0025],
               [-0.1308,  0.0565, -0.8242],
               [-0.6685, -1.1443,  1.0025],
               [-0.1308,  0.0565, -0.8242],
               [-0.6685, -1.1443,  1.0025],
               [-0.1308,  0.0565, -0.8242],
               [-0.6685, -1.1443,  1.0025],
               [-0.1308,  0.0565, -0.8242],
               [-0.6685, -1.1443,  1.0025],
               [-0.1308,  0.0565, -0.8242],
               [-0.6685, -1.1443,  1.0025],
               [-0.1308,  0.0565, -0.8242],
               [-0.6685, -1.1443,  1.0025],
               [-0.1308,  0.0565, -0.8242],
               [-0.6685, -1.1443,  1.0025]]),
    x: Batch(
           c: tensor([[-0.1607,  0.5661,  0.1415, -0.1729],
                      [-0.2221, -1.0410,  1.1426,  0.5805],
                      [-0.0153, -0.5120,  0.0779,  0.4097],
                      [-0.1607,  0.5661,  0.1415, -0.1729],
                      [-0.2221, -

In [28]:
%timeit Batch.cat(batches)

143 µs ± 429 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

334 µs ± 2.44 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
